In [12]:
import torch
from torch.nn import functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_sparse import SparseTensor

## GCNConv
&emsp;&emsp;这里贴的是教程里经过覆写后的代码。

### GCNConv类

In [46]:
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add', flow='source_to_target')
        # "Add" aggregation (Step 5).
        # flow='source_to_target' 表示消息从源节点传播到目标节点
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        adjmat = SparseTensor(row=edge_index[0], col=edge_index[1], value=torch.ones(edge_index.shape[1]))
        return self.propagate(adjmat, x=x, norm=norm, deg=deg.view((-1, 1)))

    def message(self, x_j, norm, deg_i):
        # x_j has shape [E, out_channels]
        # deg_i has shape [E, 1]
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j * deg_i

    def aggregate(self, inputs, index, ptr, dim_size):
#         print('self.aggr:', self.aggr)
#         print("`aggregate` is called")
        return super().aggregate(inputs, index, ptr=ptr, dim_size=dim_size)

#     def message_and_aggregate(self, adj_t, x, norm):
#         print('`message_and_aggregate` is called')
#         # 没有实现真实的消息传递与消息聚合的操作
#         return super().message_and_aggregate(adj_t) # NotImplementedError
        
#     def update(self, inputs, deg):
#         print(deg)
#         return inputs

### 测试

In [47]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='dataset/Cora', name='Cora')
data = dataset[0]

net = GCNConv(data.num_features, 64)
h_nodes = net(data.x, data.edge_index)

In [55]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01, weight_decay=5e-4)
net.train()

for epoch in range(200):
#     net.train()
    optimizer.zero_grad()
    
    # Get output
    out = net(data.x, data.edge_index)
    
    # Get loss
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    _, pred = out.max(dim=1)
    
    # Get predictions and calculate training accuracy
    correct = float(pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    # Backward
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data every 10 epochs
    if (epoch+1) % 10 == 0:
        net.eval()
        _, pred = net(data.x, data.edge_index).max(dim=1)
        correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        acc = correct / data.test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

[Epoch 0/200] Loss -684.7436, train acc 0.9786
[Epoch 1/200] Loss -686.4333, train acc 0.9786
[Epoch 2/200] Loss -688.1229, train acc 0.9786
[Epoch 3/200] Loss -689.8124, train acc 0.9786
[Epoch 4/200] Loss -691.5021, train acc 0.9786
[Epoch 5/200] Loss -693.1917, train acc 0.9786
[Epoch 6/200] Loss -694.8809, train acc 0.9786
[Epoch 7/200] Loss -696.5703, train acc 0.9786
[Epoch 8/200] Loss -698.2595, train acc 0.9786
[Epoch 9/200] Loss -699.9489, train acc 0.9786
Accuracy: 0.5990
[Epoch 10/200] Loss -701.6379, train acc 0.9786
[Epoch 11/200] Loss -703.3269, train acc 0.9786
[Epoch 12/200] Loss -705.0160, train acc 0.9786
[Epoch 13/200] Loss -706.7049, train acc 0.9786
[Epoch 14/200] Loss -708.3937, train acc 0.9786
[Epoch 15/200] Loss -710.0825, train acc 0.9786
[Epoch 16/200] Loss -711.7711, train acc 0.9786
[Epoch 17/200] Loss -713.4595, train acc 0.9786
[Epoch 18/200] Loss -715.1478, train acc 0.9786
[Epoch 19/200] Loss -716.8362, train acc 0.9786
Accuracy: 0.5990
[Epoch 20/200] L

[Epoch 165/200] Loss -961.8450, train acc 0.9786
[Epoch 166/200] Loss -963.5127, train acc 0.9786
[Epoch 167/200] Loss -965.1804, train acc 0.9786
[Epoch 168/200] Loss -966.8481, train acc 0.9786
[Epoch 169/200] Loss -968.5157, train acc 0.9786
Accuracy: 0.5970
[Epoch 170/200] Loss -970.1830, train acc 0.9786
[Epoch 171/200] Loss -971.8505, train acc 0.9786
[Epoch 172/200] Loss -973.5174, train acc 0.9786
[Epoch 173/200] Loss -975.1845, train acc 0.9786
[Epoch 174/200] Loss -976.8513, train acc 0.9786
[Epoch 175/200] Loss -978.5182, train acc 0.9786
[Epoch 176/200] Loss -980.1843, train acc 0.9786
[Epoch 177/200] Loss -981.8514, train acc 0.9786
[Epoch 178/200] Loss -983.5176, train acc 0.9786
[Epoch 179/200] Loss -985.1839, train acc 0.9786
Accuracy: 0.5990
[Epoch 180/200] Loss -986.8498, train acc 0.9786
[Epoch 181/200] Loss -988.5156, train acc 0.9786
[Epoch 182/200] Loss -990.1816, train acc 0.9786
[Epoch 183/200] Loss -991.8472, train acc 0.9786
[Epoch 184/200] Loss -993.5127, tra

## 作业

### 请总结MessagePassing基类的运行流程

1. 初始化，包括聚合函数、消息传递流向等（并没有去看MessagePassing类的源码，并不清楚还初始化了什么）
2. 初始化得到x和edge_index
3. 进行message和aggregate的操作

### 请复现一个一层的图神经网络的构造，总结通过继承MessagePassing基类来构造，自己的图神经网络类的规范。
&emsp;&emsp;我也不知道写的对不对。。但能够正常运行。

In [61]:
class Task2GNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(Task2GNN, self).__init__(aggr='add', flow='source_to_target')
        self.lin1 = torch.nn.Linear(in_channels, out_channels)
        self.lin2 = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x1 = self.lin1(x)
        x2 = self.lin2(x)

        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        adjmat = SparseTensor(row=edge_index[0],
                              col=edge_index[1],
                              value=torch.ones(edge_index.shape[1]))

        return self.propagate(adjmat, x1=x1, x2=x2, norm=norm, deg=deg.view((-1, 1)))
    
    def message(self, x1_j, x2_j, norm, deg_i):
        return norm.view(-1, 1) * (x1_j + x2_j) * deg_i
    
    def aggregate(self, inputs, index, ptr, dim_size):
        return super().aggregate(inputs, index, ptr=ptr, dim_size=dim_size)

In [62]:
task2gnn = Task2GNN(data.num_features, 64)
h_nodes = task2gnn(data.x, data.edge_index)

In [63]:
optimizer = torch.optim.Adam(task2gnn.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    task2gnn.train()
    optimizer.zero_grad()
    
    # Get output
    out = task2gnn(data.x, data.edge_index)
    
    # Get loss
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    _, pred = out.max(dim=1)
    
    # Get predictions and calculate training accuracy
    correct = float(pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    # Backward
    loss.backward()
    optimizer.step()
    
    # Evaluation on test data every 10 epochs
    if (epoch+1) % 10 == 0:
        task2gnn.eval()
        _, pred = task2gnn(data.x, data.edge_index).max(dim=1)
        correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
        acc = correct / data.test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

[Epoch 0/200] Loss 0.0013, train acc 0.0000
[Epoch 1/200] Loss -3.3780, train acc 0.7143
[Epoch 2/200] Loss -6.7573, train acc 0.9071
[Epoch 3/200] Loss -10.1365, train acc 0.9714
[Epoch 4/200] Loss -13.5158, train acc 0.9857
[Epoch 5/200] Loss -16.8950, train acc 1.0000
[Epoch 6/200] Loss -20.2742, train acc 1.0000
[Epoch 7/200] Loss -23.6533, train acc 1.0000
[Epoch 8/200] Loss -27.0324, train acc 1.0000
[Epoch 9/200] Loss -30.4114, train acc 1.0000
Accuracy: 0.6070
[Epoch 10/200] Loss -33.7904, train acc 1.0000
[Epoch 11/200] Loss -37.1692, train acc 1.0000
[Epoch 12/200] Loss -40.5480, train acc 1.0000
[Epoch 13/200] Loss -43.9267, train acc 1.0000
[Epoch 14/200] Loss -47.3053, train acc 1.0000
[Epoch 15/200] Loss -50.6839, train acc 1.0000
[Epoch 16/200] Loss -54.0623, train acc 1.0000
[Epoch 17/200] Loss -57.4406, train acc 1.0000
[Epoch 18/200] Loss -60.8188, train acc 1.0000
[Epoch 19/200] Loss -64.1969, train acc 1.0000
Accuracy: 0.6170
[Epoch 20/200] Loss -67.5748, train acc 

[Epoch 169/200] Loss -568.7132, train acc 1.0000
Accuracy: 0.6110
[Epoch 170/200] Loss -572.0609, train acc 1.0000
[Epoch 171/200] Loss -575.4086, train acc 1.0000
[Epoch 172/200] Loss -578.7562, train acc 1.0000
[Epoch 173/200] Loss -582.1032, train acc 1.0000
[Epoch 174/200] Loss -585.4502, train acc 1.0000
[Epoch 175/200] Loss -588.7969, train acc 1.0000
[Epoch 176/200] Loss -592.1436, train acc 1.0000
[Epoch 177/200] Loss -595.4899, train acc 1.0000
[Epoch 178/200] Loss -598.8361, train acc 1.0000
[Epoch 179/200] Loss -602.1819, train acc 1.0000
Accuracy: 0.6080
[Epoch 180/200] Loss -605.5276, train acc 1.0000
[Epoch 181/200] Loss -608.8732, train acc 1.0000
[Epoch 182/200] Loss -612.2187, train acc 1.0000
[Epoch 183/200] Loss -615.5637, train acc 1.0000
[Epoch 184/200] Loss -618.9085, train acc 1.0000
[Epoch 185/200] Loss -622.2532, train acc 1.0000
[Epoch 186/200] Loss -625.5977, train acc 1.0000
[Epoch 187/200] Loss -628.9423, train acc 1.0000
[Epoch 188/200] Loss -632.2861, tra